## Install Packages

In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('Data/Crime_Data_from_2020_to_2024.csv')
df.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [4]:
df.dtypes

DR_NO               int64
Date Rptd          object
DATE OCC           object
TIME OCC            int64
AREA                int64
AREA NAME          object
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Crm Cd Desc        object
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd         float64
Premis Desc        object
Weapon Used Cd    float64
Weapon Desc        object
Status             object
Status Desc        object
Crm Cd 1          float64
Crm Cd 2          float64
Crm Cd 3          float64
Crm Cd 4          float64
LOCATION           object
Cross Street       object
LAT               float64
LON               float64
dtype: object

## Rename the columns and remove white spaces

In [5]:
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.title()

### Data standardization 

In [6]:
Columns_Capitalized = ['Area_Name','Crm_Cd_Desc','Premis_Desc','Status_Desc','Location','Cross_Street']
for col in Columns_Capitalized:
    df[col] = df[col].str.title()

### Separate the dates and extract only the format dd/mm/yy.

In [7]:
df.Date_Rptd = df.Date_Rptd.str.split().str[0]
df.Date_Occ = df.Date_Occ.str.split().str[0]

### Remove the columns that are not relevant for analysis.

In [8]:
df.drop(['Dr_No','Area','Part_1-2','Mocodes','Premis_Cd','Weapon_Used_Cd','Status','Crm_Cd','Crm_Cd_1','Crm_Cd_2','Crm_Cd_3','Crm_Cd_4'],axis=1,inplace=True)

### Check and handle any missing values in the dataset.

In [9]:
df.isnull().sum()

Date_Rptd            0
Date_Occ             0
Time_Occ             0
Area_Name            0
Rpt_Dist_No          0
Crm_Cd_Desc          0
Vict_Age             0
Vict_Sex        124206
Vict_Descent    124216
Premis_Desc        562
Weapon_Desc     610801
Status_Desc          0
Location             0
Cross_Street    786138
Lat                  0
Lon                  0
dtype: int64

In [10]:
df.Location

0          1900 S  Longwood                     Av
1          1000 S  Flower                       St
2          1400 W  37Th                         St
3         14000    Riverside                    Dr
4                                1900    Transient
                            ...                   
932135     8700    Sharp                        Av
932136     8700    Lankershim                   Bl
932137    22700    Napa                         St
932138             Hobart                       Bl
932139     1300 W  Sunset                       Bl
Name: Location, Length: 932140, dtype: object

In [11]:
df.dropna(subset=['Vict_Sex','Premis_Desc'],inplace=True)
df.Vict_Descent.fillna('Unknown',inplace=True)

In [12]:
df.Weapon_Desc.fillna('Unknown',inplace=True)

In [13]:
df.Weapon_Desc = df.Weapon_Desc.str.replace('UNKNOWN WEAPON/OTHER WEAPON' , 'Unknown')

In [14]:
df = df[(df['Vict_Sex'] == 'M') | (df['Vict_Sex'] == 'F')]

In [15]:
# Delete all the Vict_Age that are smaller or equal to 0
df = df[df.Vict_Age > 0]

### Check and handle any duplicated values in the dataset.

In [16]:
df.duplicated().sum()

1108

In [17]:
df.drop_duplicates(inplace=True)

In [18]:
# df.Cross_Street.apply(lambda x: len(x))
# df['Cross_Street'].apply(lambda x: len(x) if isinstance(x, str) else 0)

In [19]:
# this function Combines the Location with Cross_Street if it's NAN value, 
# and otherwise it keeps the Location and get rid of the Cross_Street Where it's NAN 
def Get_Whole_Location(row):
    if pd.isna(row['Cross_Street']):
        return row['Location'].strip()
    else:
        return (row['Location'].strip() + ' ' + row['Cross_Street'].strip()).strip()
df['Whole_Location'] = df.apply(Get_Whole_Location,axis=1)

In [20]:
# delete the Location and Cross_Street columns 
df.drop(['Location', 'Cross_Street'],axis=1,inplace=True)

In [21]:
df.Vict_Descent.nunique()

21

In [22]:
Descent_Code = {
    'A' : 'Other Asian',
    'B' : 'Black',
    'C' : 'Chinese',
    'D' : 'Cambodian',
    'F' : 'Filipino',
    'G' : 'Guamanian',
    'H' : 'Hispanic/Latin/Mexican',
    'I' : 'American Indian/Alaskan Native ',
    'J' : 'Japanese',
    'K' : 'Korean',
    'L' : 'Laotian',
    'O' : 'Other',
    'P' : 'Pacific Islander',
    'S' : 'Samoan',
    'U' : 'Hawaiian',
    'V' : 'Vietnamese',
    'W' : 'White',
    'X' : 'Unknown',
    'Z' : 'Asian Indian'
 }

def Get_Vict_Ethnicity(code):
    return Descent_Code.get(code, 'Unknown')
df.Vict_Descent = df.Vict_Descent.apply(Get_Vict_Ethnicity)

In [23]:
df.Vict_Descent.nunique()

19

In [24]:
df.Vict_Descent = df.Vict_Descent.str.replace('Unknown','Other')

In [25]:
df.Vict_Sex.nunique()

2

In [26]:
df.Vict_Descent.nunique()

18

In [27]:
df.Area_Name.nunique()

21

In [28]:
df.Time_Occ = df.Time_Occ.astype(str)

In [29]:
df.head(n=2)

,Date_Rptd,Date_Occ,Time_Occ,Area_Name,Rpt_Dist_No,Crm_Cd_Desc,Vict_Age,Vict_Sex,Vict_Descent,Premis_Desc,Weapon_Desc,Status_Desc,Lat,Lon,Whole_Location
1,02/09/2020,02/08/2020,1800,Central,182,Burglary From Vehicle,47,M,Other,Bus Stop/Layover (Also Query 124),Unknown,Invest Cont,34.0444,-118.2628,1000 S Flower St
3,05/10/2023,03/10/2020,2037,Van Nuys,964,Shoplifting-Grand Theft ($950.01 & Over),19,M,Other,Clothing Store,Unknown,Invest Cont,34.1576,-118.4387,14000 Riverside Dr


In [30]:
df.Status_Desc.nunique()

6

In [31]:
df.Vict_Sex = df.Vict_Sex.map({'M':'Male','F':'Female'})

In [32]:
def Customize_Time(elem):
    if len(elem) == 1:
        elem = '0' + elem +'00'
    elif len(elem) == 2:
        elem = elem + '00'
    elif len(elem) == 3:
        elem = '0' + elem
    return elem[:2] + ':' + elem[2:]

def Time_Occurred(elem):
    if len(elem) <= 4:
        return Customize_Time(elem)
    else:
        return elem
df['Time_Occ'] = df['Time_Occ'].apply(Time_Occurred)
df.Time_Occ

1         18:00
3         20:37
4         12:00
5         23:00
6         09:00
          ...  
932134    14:15
932135    19:20
932136    07:20
932137    23:30
932138    15:10
Name: Time_Occ, Length: 685625, dtype: object

In [33]:
# Hours_Gt_24 is fuction that checks if the hours are greater then 24, if so then we assign it 0 in order to delete them later
def Hours_Gt_24(elem):
    test_cte = pd.to_numeric(elem[:2])
    if test_cte < 24 :
        return elem
    else:
        elem = 0
        return elem

df.Time_Occ = df.Time_Occ.apply(Hours_Gt_24)

In [34]:
df = df[df.Time_Occ != 0]
df.shape

(676231, 15)

In [35]:
df.Status_Desc.unique()

array(['Invest Cont', 'Juv Arrest', 'Adult Arrest', 'Adult Other',
       'Juv Other', 'Unk'], dtype=object)

In [36]:
var = {'Invest Cont':'Investigation Continued','Juv Arrest':'Juvenile Arrest','Juv Other':'Juvenile Other','Unk':'Unknown'}
df.Status_Desc.replace(var,inplace=True)

### Calculate the Response Time 

#### First Step: Extract the month, day, and year from "Date_Rptd" and "Date_OCc"

In [37]:
Df_Date_Test = df.copy()
Df_Date_Test['Rptd_Month'] = Df_Date_Test['Date_Rptd'].str.split('/').str[0]
Df_Date_Test['Rptd_Day'] = Df_Date_Test['Date_Rptd'].str.split('/').str[1]
Df_Date_Test['Rptd_Year'] = Df_Date_Test['Date_Rptd'].str.split('/').str[2]

Df_Date_Test['Occ_Month'] = Df_Date_Test['Date_Occ'].str.split('/').str[0]
Df_Date_Test['Occ_Day'] = Df_Date_Test['Date_Occ'].str.split('/').str[1]
Df_Date_Test['Occ_Year'] = Df_Date_Test['Date_Occ'].str.split('/').str[2]
Df_Date_Test.head(n=2)

,Date_Rptd,Date_Occ,Time_Occ,Area_Name,Rpt_Dist_No,Crm_Cd_Desc,Vict_Age,Vict_Sex,Vict_Descent,Premis_Desc,...,Status_Desc,Lat,Lon,Whole_Location,Rptd_Month,Rptd_Day,Rptd_Year,Occ_Month,Occ_Day,Occ_Year
1,02/09/2020,02/08/2020,18:00,Central,182,Burglary From Vehicle,47,Male,Other,Bus Stop/Layover (Also Query 124),...,Investigation Continued,34.0444,-118.2628,1000 S Flower St,02,09,2020,02,08,2020
3,05/10/2023,03/10/2020,20:37,Van Nuys,964,Shoplifting-Grand Theft ($950.01 & Over),19,Male,Other,Clothing Store,...,Investigation Continued,34.1576,-118.4387,14000 Riverside Dr,05,10,2023,03,10,2020


#### Second Step: Verify that the crime was not reported before it's occured

In [38]:
Df_Date_Test[Df_Date_Test['Rptd_Year'] < Df_Date_Test['Occ_Year']]

,Date_Rptd,Date_Occ,Time_Occ,Area_Name,Rpt_Dist_No,Crm_Cd_Desc,Vict_Age,Vict_Sex,Vict_Descent,Premis_Desc,...,Status_Desc,Lat,Lon,Whole_Location,Rptd_Month,Rptd_Day,Rptd_Year,Occ_Month,Occ_Day,Occ_Year


In [39]:
Df_Date_Test[(Df_Date_Test['Rptd_Year'] == Df_Date_Test['Occ_Year']) & (Df_Date_Test['Rptd_Month'] < Df_Date_Test['Occ_Month'])]

,Date_Rptd,Date_Occ,Time_Occ,Area_Name,Rpt_Dist_No,Crm_Cd_Desc,Vict_Age,Vict_Sex,Vict_Descent,Premis_Desc,...,Status_Desc,Lat,Lon,Whole_Location,Rptd_Month,Rptd_Day,Rptd_Year,Occ_Month,Occ_Day,Occ_Year


In [40]:
Df_Date_Test[(Df_Date_Test['Rptd_Year'] == Df_Date_Test['Occ_Year']) & (Df_Date_Test['Rptd_Month'] == Df_Date_Test['Occ_Month']) & (Df_Date_Test['Rptd_Day'] < Df_Date_Test['Occ_Day'])]

,Date_Rptd,Date_Occ,Time_Occ,Area_Name,Rpt_Dist_No,Crm_Cd_Desc,Vict_Age,Vict_Sex,Vict_Descent,Premis_Desc,...,Status_Desc,Lat,Lon,Whole_Location,Rptd_Month,Rptd_Day,Rptd_Year,Occ_Month,Occ_Day,Occ_Year


In [41]:
df = df.rename(columns={'Whole_Location':'Location'})
# df.columns

In [42]:
# df.Location
df = df[(df.Lat != 00.00) & (df.Lon != 00.00)]

In [43]:
# df.to_csv('Data/Cleaned_Data.csv',index=False)